In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from random import seed
from random import randint
import random
import plotly.express as px

In [250]:
df = pd.read_csv('EURUSD.csv')
df['time'] = pd.to_datetime(df.timestamp, unit='ms')
del(df['timestamp'])

In [251]:
df=df[4000000:4500000]

In [247]:
class Position :
    def __init__(self,open_time,open_price,position,takeprofit,lot_size,stop,ticket):
        self.open_time=open_time
        self.ticket=ticket
        self.open_price=open_price
        self.lot_size=lot_size
        self.position=position
        self.stop=stop
        self.close_price=0
        self.close_time=0
        self.takeprofit=takeprofit
        self.openclose=1
        self.activator=0
        if position=='buy':
            self.mul=-1
        if position =='sell':
            self.mul=1
        self.pl=0.0
        self.signal=0
        self.calculatedpl=0
    def close_position(self,close_time,close_price,ratio):
        self.close_time=close_time
        self.close_price=close_price
        lot_asked=ratio*self.lot_size/100
        self.lot_size=self.lot_size-lot_asked
        self.pl=self.pl+(self.open_price-self.close_price)*self.mul*lot_asked-(lot_asked*0.00015)
        self.calculatedpl=1
    def as_dict(self):
        return {'open_price':self.open_price,'open_time':self.open_time,'close_time':self.close_time,
                'close_price': self.close_price,'lot_size':self.lot_size,'pl':self.pl,'mul':self.mul}

In [276]:
open_positions=[]
position_history=[]
lot_size=1
ticket1win=0
ticket2win=0
ticket3win=0
ticket4win=0
ticketlossfinal=0
ticketlossnf=0
winar=0
for i in df.index:
    if open_positions==[]:
        if i-2 >=4000000 :
            s=random.randint(0,1)
            if s==1 :
                open_positions.append(Position(df['time'][i],df['close'][i],'buy',df['close'][i]+0.0020,lot_size,df['close'][i]-0.001,1))
            elif s==0 :
                open_positions.append(Position(df['time'][i],df['close'][i],'sell',df['close'][i]-0.0020,lot_size,df['close'][i]+0.001,1))
    else :
        for p in range(len(open_positions)):
            if open_positions[p].position=='buy' and open_positions[p].ticket==1:
                if df['high'][i]>=open_positions[p].takeprofit :
                    open_positions[p].close_position(df['time'][i],open_positions[p].takeprofit,100)
                    position_history.append(open_positions[p])
                    
                    #DONE HERE
                    for ll in range(len(open_positions)):
                        if open_positions[ll].ticket==2:
                            open_positions.append(Position(df['time'][i],open_positions[p].takeprofit,'sell',open_positions[p].takeprofit-0.0040,lot_size,open_positions[p].takeprofit+0.004,3))
                            open_positions[ll].activator=1
                            ticket1win=ticket1win-1
                            ticket2win=ticket2win+1
                    del(open_positions[p])
                    ticket1win=ticket1win+1
                    break
                elif df['low'][i]<=open_positions[p].stop and open_positions[p].activator==0:
                        open_positions.append(Position(df['time'][i],open_positions[p].stop,'sell',open_positions[p].stop-0.0020,lot_size,open_positions[p].stop+0.0040,2))
                        
                        open_positions[p].activator=1
                        break
            elif open_positions[p].position=='sell' and open_positions[p].ticket==1:
                if df['low'][i]<=open_positions[p].takeprofit :
                    open_positions[p].close_position(df['time'][i],open_positions[p].takeprofit,100)
                    position_history.append(open_positions[p])
                    
                    #DONE HERE
                    for ll in range(len(open_positions)):
                        if open_positions[ll].ticket==2:
                            open_positions.append(Position(df['time'][i],open_positions[p].takeprofit,'buy',open_positions[p].takeprofit+0.0040,lot_size,open_positions[p].takeprofit-0.004,3))
                            open_positions[ll].activator=1
                            ticket1win=ticket1win-1
                            ticket2win=ticket2win+1
                    del(open_positions[p])
                    ticket1win=ticket1win+1
                    break
                elif df['high'][i]>=open_positions[p].stop and open_positions[p].activator==0:
                        open_positions.append(Position(df['time'][i],open_positions[p].stop,'buy',open_positions[p].stop+0.0020,lot_size,open_positions[p].stop-0.0040,2))

                        open_positions[p].activator=1
                        break
            elif open_positions[p].position=='buy' and open_positions[p].ticket==2 and open_positions[p].activator==0:
                if df['high'][i]>=open_positions[p].takeprofit :
                    open_positions[p].close_position(df['time'][i],open_positions[p].takeprofit,100)
                    position_history.append(open_positions[p])
                    #DONE HERE
                    
                    ticket2win=ticket2win+1
                    open_positions.append(Position(df['time'][i],open_positions[p].takeprofit,'sell',open_positions[p].takeprofit-0.00400,lot_size,open_positions[p].takeprofit+0.0040,3))
                  
                    del(open_positions[p])
                    
                    break
                elif df['low'][i]<=open_positions[p].stop and open_positions[p].activator==0:
                    open_positions[p].activator=1
                    open_positions.append(Position(df['time'][i],open_positions[p].stop,'sell',open_positions[p].stop-0.00400,lot_size,open_positions[p].stop+0.0040,3))
                    break
                
            elif open_positions[p].position=='sell' and open_positions[p].ticket==2 and open_positions[p].activator==0:
                if df['low'][i]<=open_positions[p].takeprofit :
                    open_positions[p].close_position(df['time'][i],open_positions[p].takeprofit,100)
                    position_history.append(open_positions[p])
                    open_positions[p].activator=1
                    open_positions.append(Position(df['time'][i],open_positions[p].takeprofit,'buy',open_positions[p].takeprofit+0.0040,lot_size,open_positions[p].takeprofit-0.0040,3))
                    #DONE HERE
                    
                    del(open_positions[p])
                    ticket2win=ticket2win+1
                    
                    break
                elif df['high'][i]>=open_positions[p].stop and open_positions[p].activator==0:
                    open_positions[p].activator=1
                    open_positions.append(Position(df['time'][i],open_positions[p].stop,'buy',open_positions[p].stop+0.00400,lot_size,open_positions[p].stop-0.0040,3))
                    break
            elif open_positions[p].position=='buy' and open_positions[p].ticket==3:
                if df['high'][i]>=open_positions[p].takeprofit :
                    for ps in range(len(open_positions)):
                        open_positions[ps].close_position(df['time'][i],open_positions[p].takeprofit,100)
                        position_history.append(open_positions[ps])
                        
                    open_positions=[]
                    ticket3win=ticket3win+1
                    winar=winar+1
                    break
                  
                elif df['low'][i]<=open_positions[p].stop and open_positions[p].activator==0:
                    open_positions[p].close_position(df['time'][i],open_positions[p].stop,100)
                    position_history.append(open_positions[p])
                    open_positions.append(Position(df['time'][i],open_positions[p].stop,'buy',open_positions[p].stop+0.00800,lot_size,open_positions[p].stop-0.0080,4))
                    del(open_positions[p])
                    ticketlossnf=ticketlossnf+1
                    break
                    
            elif open_positions[p].position=='sell' and open_positions[p].ticket==3:
                if df['low'][i]<=open_positions[p].takeprofit :
                    for ps in range(len(open_positions)):
                        open_positions[ps].close_position(df['time'][i],open_positions[p].takeprofit,100)
                        position_history.append(open_positions[ps])
                        
                    open_positions=[]
                    ticket3win=ticket3win+1
                    winar=winar+1
                    break
                elif df['high'][i]>=open_positions[p].stop and open_positions[p].activator==0:
                    open_positions[p].close_position(df['time'][i],open_positions[p].stop,100)
                    position_history.append(open_positions[p])
                    
                    open_positions.append(Position(df['time'][i],open_positions[p].stop,'sell',open_positions[p].stop-0.00800,lot_size,open_positions[p].stop+0.0080,4))
                    del(open_positions[p])
                    ticketlossnf=ticketlossnf+1
                    
                    break
            elif open_positions[p].position=='buy' and open_positions[p].ticket==4:
                if df['high'][i]>=open_positions[p].takeprofit :
                    for ps in range(len(open_positions)):
                        open_positions[ps].close_position(df['time'][i],open_positions[p].takeprofit,100)
                        position_history.append(open_positions[ps])
                        
                    open_positions=[]
                    ticket4win=ticket4win+1
                    break
                  
                elif df['low'][i]<=open_positions[p].stop :
                    for ps in range(len(open_positions)):
                        open_positions[ps].close_position(df['time'][i],open_positions[p].stop,100)
                        position_history.append(open_positions[ps])
                        ####DOOONE
                        
                    open_positions=[]
                    ticketlossfinal=ticketlossfinal+1
                    break
            elif open_positions[p].position=='sell' and open_positions[p].ticket==4:
                if df['low'][i]<=open_positions[p].takeprofit :
                    for ps in range(len(open_positions)):
                        open_positions[ps].close_position(df['time'][i],open_positions[p].takeprofit,100)
                        position_history.append(open_positions[ps])
                        #####DOOONE
                        
                    open_positions=[]
                    ticket4win=ticket4win+1
                    break
                elif df['high'][i]>=open_positions[p].stop:
                    for ps in range(len(open_positions)):
                        open_positions[ps].close_position(df['time'][i],open_positions[p].stop,100)
                        position_history.append(open_positions[ps])
                        
                        #####DOOONE
                    open_positions=[]
                    ticketlossfinal=ticketlossfinal+1
                    break
ticket1win=ticket1win+1
print("ticket 1 win ")
print (ticket1win)
print("ticket 2 win ")
print (ticket2win)
print("ticket 3 win ")
print (ticket3win)
print("ticket 4 win ")
print (ticket4win)
print("ticket loss final ")
print(ticketlossfinal)
print("ticket loss not final ")
print(ticketlossnf)
print("winar")
print(winar)

ticket 1 win 
241
ticket 2 win 
399
ticket 3 win 
182
ticket 4 win 
104
ticket loss final 
112
ticket loss not final 
217
winar
182


In [277]:
sss=position_history
pos_df=pd.DataFrame([p.as_dict() for p in sss])
pos_df['cumulative_profit']=pos_df["pl"].cumsum()
fig_pos=px.line(pos_df,x='close_time',y='cumulative_profit')
fig_pos.show()

In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

E:\Users\azar\AppData\Local\Temp\ipykernel_12160\3777615979.py:1: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display

